In [ ]:
import pandas as pd

# Data Biases

In [ ]:
from utils.sql import get_ratings_df
df_ratings = get_ratings_df()

In [ ]:
df_user = pd.read_json('user_data_full.json')
df_movie = pd.read_json('movie_data_full.json')

In [ ]:
df_ratings['user_id'] = df_ratings['user_id'].astype(int)

In [ ]:
df = df_ratings.merge(df_user, on='user_id', how='right').merge(df_movie, left_on='movie_id', right_on='id', how='right')
df['age_range'] = df['age'].apply(lambda x: f'{int(((x-1)//10)*10)}-{int(((x-1)//10)*10+9)}' if not pd.isnull(x) else None)
df = df[['user_id', 'movie_id', 'rating', 'age', 'age_range', 'occupation', 'gender',
         'adult', 'genres', 'release_date', 'spoken_languages']]

g = df['genres'].apply(lambda lst: [d['name'] for d in lst]).str.join('|')
genre_dummies = g.str.get_dummies(sep='|')
df = pd.concat([df, genre_dummies], axis=1).drop(columns='genres')

l = df['spoken_languages'].apply(lambda lst: [d['iso_639_1'] for d in lst]).str.join('|')
language_dummies = l.str.get_dummies(sep='|')
df = pd.concat([df, language_dummies], axis=1).drop(columns='spoken_languages')

In [ ]:
df.groupby('gender').apply(
    lambda x: (x[genre_dummies.columns].mean() * 100).sort_values(ascending=False).head(10)
).reset_index().rename(columns={0: 'pct'}).pivot_table(values='pct', index='level_1', columns='gender').sort_values('F', ascending=False)

In [ ]:
df.groupby('age_range').apply(
    lambda x: (x[genre_dummies.columns].mean() * 100).sort_values(ascending=False).head(10)
).reset_index().rename(columns={0: 'pct'}).pivot_table(values='pct', index='level_1', columns='age_range').sort_values('20-29', ascending=False)

In [ ]:
(df[genre_dummies.columns].multiply(df['rating'], axis=0).sum()/df[genre_dummies.columns].sum()).sort_values(ascending=False)

In [ ]:
df.groupby('gender').size()/len(df)

In [ ]:
df.groupby('age_range').size()/len(df)

# Fairness Analysis

In [ ]:
from utils.sql import get_engine
from sqlalchemy import text
engine = get_engine()

with engine.begin() as con:
    query = '''
    SELECT * FROM recommend WHERE timestamp BETWEEN (
        SELECT MIN(timestamp) FROM watch
    ) AND (
        SELECT MAX(timestamp) FROM watch
    )
    '''
    df_recommend = pd.read_sql_query(text(query), con)
    df_watch = pd.read_sql_table('watch', con)


In [ ]:
df_watch['timestamp']   = pd.to_datetime(df_watch['timestamp'])
df_recommend['timestamp'] = pd.to_datetime(df_recommend['timestamp'])

df_recommend = df_recommend.reset_index().rename(columns={'index':'rec_idx'})

df_recommend['rec_list'] = (
    df_recommend['recommendations']
    .str.split(', ')
    .apply(lambda lst: [x for x in lst])
)

df_rec = (
    df_recommend[['rec_idx','user_id','timestamp','rec_list']]
    .explode('rec_list')
    .rename(columns={'timestamp':'rec_ts', 'rec_list':'movie_id'})
)

df_merge = pd.merge(
    df_rec,
    df_watch.rename(columns={'timestamp':'watched_ts'}),
    on=['user_id','movie_id'],
    how='inner'
)

df_success = df_merge[
    (df_merge['watched_ts'] >= df_merge['rec_ts']) &
    (df_merge['watched_ts'] <= df_merge['rec_ts'] + pd.Timedelta(hours=24))
]

success_flags = (
    df_success[['rec_idx']]
    .drop_duplicates()
    .assign(success=1)
)

df_recommend = (
    df_recommend
    .merge(success_flags, on='rec_idx', how='left')
    .fillna({'success': 0})
)

df_recommend['success'] = df_recommend['success'].astype(int)

df_recommend = df_recommend[['user_id', 'success']]
df_recommend['user_id'] = df_recommend['user_id'].astype(int)
df_recommend = df_recommend.merge(df_user, on='user_id')
df_recommend['age_range'] = df_recommend['age'].apply(lambda x: f'{int(((x-1)//10)*10)}-{int(((x-1)//10)*10+9)}' if not pd.isnull(x) else None)


In [ ]:
df_recommend.groupby('success').size()

In [ ]:
df_recommend.groupby('gender')[['success']].mean()

In [ ]:
df_recommend.groupby('age_range')[['success']].mean()